# Flood Model Training Notebook

Train a Flood ConvLSTM Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2

import pathlib
import logging

import tensorflow as tf

from usl_models.flood_ml import constants
from usl_models.flood_ml.model import FloodModel
from usl_models.flood_ml import dataset
from usl_models.flood_ml import visualizer
from usl_models.flood_ml import eval


logging.getLogger().setLevel(logging.INFO)

sim_names = ["Manhattan-config_v1/Rainfall_Data_1.txt"]
filecache_dir = pathlib.Path("/home/shared/climateiq/filecache")

In [ ]:
# @title Download dataset (only needs to be run the first time)
# dataset.download_dataset(sim_names, dataset_splits=["train"], output_path=filecache_dir)

In [ ]:
# Load dataset and train model.
train_dataset = dataset.load_dataset_windowed_cached(
    filecache_dir=filecache_dir,
    sim_names=sim_names,
    batch_size=4,
    dataset_split="train",
)
model = FloodModel(
    FloodModel.Params(
        lstm_units=32,
        lstm_kernel_size=3,
        lstm_dropout=0.2,
        lstm_recurrent_dropout=0.2,
        m_rainfall=6,
        n_flood_maps=5,
        num_features=22,
    )
)
model.fit(train_dataset, epochs=1)

In [ ]:
# Test calling the model on some data.
inputs, labels_ = next(iter(train_dataset))
prediction = model.call(inputs)
prediction.shape

In [ ]:
# Test calling the model for n predictions
full_dataset = dataset.load_dataset_cached(
    filecache_dir=filecache_dir, sim_names=sim_names, batch_size=4
)
inputs, labels, metadata_ = next(iter(full_dataset))
predictions = model.call_n(inputs, n=19)
predictions.shape, labels.shape

In [ ]:
spatial_maes = []
temporal_mae, temporal_rmse = [], []
max_mae = 0.0
max_spatial_mae = None
highest_error_nse = None
highest_error_pred = None
highest_error_label = None

for prediction, label in zip(tf.unstack(predictions), tf.unstack(labels)):
    max_pred = tf.reduce_max(prediction, axis=0)
    max_label = tf.reduce_max(label, axis=0)
    spatial_mae = eval.spatial_mae(max_pred, max_label)

    temporal_mae.append(eval.temporal_mae(prediction, label))
    temporal_rmse.append(eval.temporal_rmse(prediction, label))

    spatial_maes.append(spatial_mae)

    mae = tf.reduce_mean(spatial_mae)
    if mae > max_mae:
        max_mae = mae
        max_spatial_mae = spatial_mae
        max_mae_nse = eval.spatial_nse(prediction, label)
        max_mae_pred = max_pred
        max_mae_label = max_label

num_test_examples = len(spatial_maes)
overall_mae = tf.reduce_mean(tf.stack(spatial_maes))
temporal_mae = tf.reduce_mean(tf.stack(temporal_mae), axis=0)
temporal_rmse = tf.reduce_mean(tf.stack(temporal_rmse), axis=0)

In [ ]:
visualizer.plot_maps(
    "Test",
    spatial_mae=max_spatial_mae,
    nse=max_mae_nse,
    pred=max_mae_pred,
    label=max_mae_label,
)